In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import allennlp
import torch

In [35]:
from allennlp.data.dataset_readers import Seq2SeqDatasetReader
from allennlp.data.tokenizers import WordTokenizer
from allennlp.data.tokenizers.word_splitter import JustSpacesWordSplitter 
from allennlp.data import Vocabulary
from allennlp.modules import TokenEmbedder
from allennlp.modules import Embedding
from allennlp.common.params import Params
from allennlp.data.iterators import BasicIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors.simple_seq2seq import SimpleSeq2SeqPredictor
from allennlp.nn.beam_search import BeamSearch
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper

from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.dataset import Batch
from allennlp.modules.attention.dot_product_attention import DotProductAttention

from rnn2rnn import Rnn2Rnn
from seq2prob import Seq2Prob
from data_utils import *


In [4]:
torch.manual_seed(1)

In [5]:
reader = Seq2SeqDatasetReader(source_tokenizer=WordTokenizer(word_splitter=JustSpacesWordSplitter()),
                              source_token_indexers={"ids": SingleIdTokenIndexer(namespace="language_A")},
                              target_token_indexers={"ids": SingleIdTokenIndexer(namespace="language_B")},
                                                     source_add_start_token=True)

In [6]:
train_dataset = reader.read(file_path="toy_data/train.txt")
validation_dataset = reader.read(file_path="toy_data/dev.txt")

100000it [00:08, 11937.68it/s]
1000it [00:00, 15409.87it/s]


In [7]:
vocab = Vocabulary.from_instances(train_dataset)

100%|██████████| 100000/100000 [00:02<00:00, 47834.80it/s]


In [8]:
"""Generators take in one-hot encoded vectors (or distribution), and output the same format."""

'Generators take in one-hot encoded vectors (or distribution), and output the same format.'

# Create stuff

Hparams

---

In [61]:
from build_modules import build_modules

In [63]:
modules_dict = build_modules(vocab)

In [10]:
iterator = BasicIterator(batch_size=BATCH_SIZE)
iterator.index_with(vocab)
generator_train = iterator(train_dataset, shuffle=False)
generator_validation = iterator(validation_dataset, shuffle=False)

In [43]:
curr_batch = sample_new_batch(generator_validation, vocab)
model_input = prepare_model_input(source_tokens_dict=curr_batch['source_tokens'])
model_output = src2tgt.forward(**model_input)
model_input = prepare_model_input(source_tokens_dict=model_output)
model_output = tgt2src.forward(**model_input, target_tokens=curr_batch['source_tokens'])

In [54]:
modules_dict["discriminator_a"](**model_input).mean()

tensor(0.4861, grad_fn=<MeanBackward1>)

In [58]:
discriminator_b(**model_input).mean()

tensor(0.4713, grad_fn=<MeanBackward1>)

In [32]:
model_output['loss']

tensor(2.5631, grad_fn=<DivBackward1>)

In [ ]:
optimizer = torch.optim.SGD(src2tgt.parameters(), lr=0.1)

In [ ]:
trainer = Trainer(model=src2tgt, 
                  optimizer=optimizer, 
                  iterator=iterator, 
                  train_dataset=train_dataset, 
                  validation_dataset=validation_dataset, 
                  patience=3, 
                  num_epochs=1000)

In [ ]:
trainer.train()

In [ ]:
src2tgt.training=False

In [ ]:
predictor = SimpleSeq2SeqPredictor(src2tgt, dataset_reader=reader)

In [ ]:
vocab.get_token_index(namespace="target_tokens", token="@end@")

In [ ]:
src2tgt._beam_search = BeamSearch(end_index=vocab.get_token_index(namespace="target_tokens", token="@end@"), 
                                  max_steps=20, 
                                  beam_size=5)

In [ ]:
vocab.get_token_from_index(namespace="target_tokens", index=8)

In [ ]:
predictor.predict("6 7 3 2 1 5 9 2 7 7")